# Importing data

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

In [3]:
engine = create_engine('sqlite:///database.sqlite')

In [4]:
inspector = inspect(engine)
tables = inspector.get_table_names()
tables

['Ball_by_Ball',
 'Batsman_Scored',
 'Batting_Style',
 'Bowling_Style',
 'City',
 'Country',
 'Extra_Runs',
 'Extra_Type',
 'Match',
 'Out_Type',
 'Outcome',
 'Player',
 'Player_Match',
 'Rolee',
 'Season',
 'Team',
 'Toss_Decision',
 'Umpire',
 'Venue',
 'Wicket_Taken',
 'Win_By',
 'sysdiagrams']

# Tasks

In [6]:
#  Select All Columns from Player’s Table
df = pd.read_sql_query('SELECT * FROM Player', engine)

df

,Player_Id,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
0,1,SC Ganguly,1972-07-08 00:00:00,1,1.0,1
1,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4
2,3,RT Ponting,1974-12-19 00:00:00,2,1.0,5
3,4,DJ Hussey,1977-07-15 00:00:00,2,2.0,5
4,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6
...,...,...,...,...,...,...
464,465,DL Chahar,1992-08-07 00:00:00,2,1.0,1
465,466,P Dharmani,1974-09-27 00:00:00,2,NaN,1
466,467,RV Pawar,1979-09-06 00:00:00,1,7.0,1
467,468,KH Devdhar,1989-12-14 00:00:00,2,NaN,1


In [8]:
# a SQL query to calculate the total runs scored by each batsman

df = pd.read_sql(
    '''SELECT player_name, SUM(runs) AS total_runs 
    FROM Player 
    GROUP BY player_name 
    ORDER BY total_runs DESC''',
    engine
)

df

OperationalError: (sqlite3.OperationalError) no such column: runs
[SQL: SELECT player_name, SUM(runs) AS total_runs 
    FROM Player 
    GROUP BY player_name 
    ORDER BY total_runs DESC]
(Background on this error at: https://sqlalche.me/e/20/e3q8)